In [4]:
import seaborn as sns
sns.set()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
train_data = pd.read_csv('../input/salespredictiondatasetsahan/sales_train.csv')
test_data = pd.read_csv('../input/salespredictiondatasetsahan/test.csv')

In [6]:
train_data.head()

In [7]:
test_data.head()

In [8]:
train_Data = train_data.copy()
train_Data.isna().sum()

In [9]:
train_Data = train_Data.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')

In [10]:
train_Data.head()

In [11]:
train_Data.columns

In [12]:
test_Data = test_data.copy()
test_Data = test_Data.pivot_table(index = ['shop_id','item_id'],fill_value = 0)
Combine_train_test = pd.merge(test_Data, train_Data, how = 'left', on = ['shop_id','item_id']).fillna(0)

In [13]:
Combine_train_test = Combine_train_test.sort_values(by = 'ID')

In [14]:
Combine_train_test.head(10)

In [15]:
train_data.shape

In [16]:
Combine_train_test = Combine_train_test.drop(columns = ['ID'])

train_data = np.array(Combine_train_test.values[:,:-1]).reshape(np.array(Combine_train_test.values[:,:-1]).shape[0],np.array(Combine_train_test.values[:,:-1]).shape[1], 1)

train_target = Combine_train_test.values[:,-1:]

test_data = np.array(Combine_train_test.values[:,1:]).reshape(np.array(Combine_train_test.values[:,1:]).shape[0],np.array(Combine_train_test.values[:,1:]).shape[1], 1)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, TimeDistributed, Flatten

In [18]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (train_data.shape[1], 1)))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50))
model.add(Dense(25))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer = 'adam')

In [19]:
history_1 = model.fit(train_data, train_target, epochs = 15, batch_size = 128)
df_his1 = pd.DataFrame(history_1.history)

In [20]:
plt.figure(figsize = (10 , 10))
sns.lineplot(x= df_his1.index + 1, y = df_his1['loss'], color='r', label="Range loss")

In [21]:
submission = pd.read_csv('../input/salespredictiondatasetsahan/sample_submission.csv')
submission['item_cnt_month'] = model.predict(test_data)
submission['item_cnt_month'] = submission['item_cnt_month'].clip(0,30)
submission.to_csv('submission.csv', index=False)

In [22]:
submission